In [1]:
# Import Dependencies
import psycopg2
import pandas as pd

# Import postgres server info key and BEA API Key
from postgres_info import user, password

In [2]:
# Import data into pandas dataframe
global_warming_df = pd.read_csv('Resources/global_warming_dataset.csv')
global_warming_df.head()

,Country,Year,Temperature_Anomaly,CO2_Emissions,Population,Forest_Area,GDP,Renewable_Energy_Usage,Methane_Emissions,Sea_Level_Rise,...,Waste_Management,Per_Capita_Emissions,Industrial_Activity,Air_Pollution_Index,Biodiversity_Index,Ocean_Acidification,Fossil_Fuel_Usage,Energy_Consumption_Per_Capita,Policy_Score,Average_Temperature
0,Country_103,1913,-1.163537,8.876061e+08,1.627978e+08,54.872178,6.139887e+12,76.710013,8.317626e+06,8.111839,...,82.691409,2.285351,4.060975,150.285539,90.073356,8.025470,39.163860,1480.164332,78.870012,20.825292
1,Country_180,1950,-0.432122,4.497517e+08,4.281359e+08,84.051006,2.601447e+12,68.450021,6.206540e+06,42.025915,...,59.322883,17.411668,85.300604,27.305922,88.289837,8.021719,28.252554,1482.730048,32.600905,28.720587
2,Country_93,2014,0.444954,4.579080e+08,4.926732e+08,72.295357,5.192677e+12,36.725699,1.056885e+06,20.953840,...,94.982931,12.039703,83.804880,216.911429,86.936256,7.647408,61.548382,706.918809,37.671300,15.014084
3,Country_15,2020,-1.171616,5.049503e+08,1.252169e+09,17.259684,8.252128e+12,77.547901,1.986813e+06,45.599595,...,62.064250,2.853957,47.014265,35.869182,44.904331,7.569353,82.423750,2616.238324,86.581725,-1.277086
4,Country_107,1964,-0.564038,6.898891e+08,2.932960e+08,44.438605,8.560746e+12,10.019576,3.313252e+06,7.652150,...,84.431279,19.801173,89.379613,284.263093,8.102916,8.015415,29.964450,4975.683780,20.618406,2.861989


In [3]:
# Convert dataframe values to a list
global_warming_list = global_warming_df.values.tolist()

In [4]:
# Use a 'try' statement to handle errors connecting to the database
try:
    
    # Connect to the database
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=user,
        password=password,
        database="postgres"
    )
    
    # Make sure autocommit is 'True' to commit new database successfully
    conn.autocommit = True

# Use 'except' statement to handle and display an error connect to PostgreSQL
except psycopg2.Error as e:
    print('-'*60)
    print("Error connecting to the database:")
    print(e)

# Use 'else' statemnt is connection if successful
else:
    print('-'*60)
    print("Connection established successfully")

# Create cursor
cur = conn.cursor()

# Query to create 'ev_db' database
sql = '''CREATE DATABASE global_warming_db;'''

# Use 'try' statement to execute above SQL statement
print('-'*60)
try:
    cur.execute(sql)
    print("Database successfully created!")

# Use 'except' statement to handle if database already exists
except:
    print("Database already exists! Execute command aborted...")

# Close cursor
cur.close()

# Close connection
conn.close()

------------------------------------------------------------
Connection established successfully
------------------------------------------------------------
Database already exists! Execute command aborted...


In [5]:
# Use a 'try' statement to handle errors connecting to the database
try:

    # Connect to the database
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=user,
        password=password,
        database="global_warming_db"
    )

# Use 'except' statement to handle and display an error connect to PostgreSQL
except psycopg2.Error as e:
    print('-'*60)
    print("Error connecting to the database:")
    print(e)

# Use 'else' statemnt if connection is successful
else:
    print('-'*60)
    print("Connection re-established successfully")

# Create cursor
cur = conn.cursor() 

# SQL queries to drop tables if already exists. Use 'CASCADE' to ensure all dependent tables are dropped
drop_table_query = '''DROP TABLE IF EXISTS global_warming_data CASCADE;'''

# SQL queries to create tables for our data
create_table_query = '''
    CREATE TABLE global_warming_data (
        record_id SERIAL PRIMARY KEY,
        country VARCHAR(12),
        year INT,
        temperature_anomaly FLOAT,
        co2_emissions FLOAT,
        population FLOAT,
        forest_area FLOAT,
        gdp FLOAT,
        renewable_energy_usage FLOAT,
        methane_emissions FLOAT,
        sea_level_rise FLOAT,
        artic_ice_extent FLOAT,
        urbanization FLOAT,
        deforestation_rate FLOAT,
        extreme_weather_events INT,
        average_rainfall FLOAT,
        solar_energy_potential FLOAT,
        waste_management FLOAT,
        per_capita_emissions FLOAT,
        industrial_activity FLOAT,
        air_pollution_index FLOAT,
        biodiversity_index FLOAT,
        ocean_acidification FLOAT,
        fossil_fuel_usage FLOAT,
        energy_consumption_per_capita FLOAT,
        policy_score FLOAT,
        average_temperature FLOAT
    );
    '''

# Execute Drop table queries
cur.execute(drop_table_query)

# Execute the CREATE TABLE queries
cur.execute(create_table_query)
print('-'*60)
print("Table created or recreated successfully!")

------------------------------------------------------------
Connection re-established successfully
------------------------------------------------------------
Table created or recreated successfully!


In [6]:
# Print statement showing data is loading
print('-'*60)
print('Loading global warming data into table...')

# Use a for loop to insert each row of data into main vehicle table
for row in global_warming_list:
    sql = '''INSERT INTO global_warming_data 
    (country, year, temperature_anomaly, co2_emissions, population, forest_area, gdp, renewable_energy_usage, methane_emissions, 
    sea_level_rise, artic_ice_extent, urbanization, deforestation_rate, extreme_weather_events, average_rainfall, solar_energy_potential, 
    waste_management, per_capita_emissions, industrial_activity, air_pollution_index, biodiversity_index, ocean_acidification, fossil_fuel_usage, 
    energy_consumption_per_capita, policy_score, average_temperature) 
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
    cur.execute(sql, row)

# Print statment showing data loaded successfully
print('Gloabal Warming data added successfully!')

# Commit all the changes to the database
conn.commit()

# Close the cursor
cur.close()

# Close the connection
conn.close()

------------------------------------------------------------
Loading global warming data into table...
Gloabal Warming data added successfully!


# Data Extraction and Processing

In [7]:
# Use a 'try' statement to attempt to conenct to database and to handle errors that occur
try:
    # Connect to the database
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=user,
        password=password,
        database="global_warming_db"
    )

# Use an 'except' block to handle error connecting to database    
except psycopg2.Error as e:
    print("Error connecting to the database:")
    print(e)

# Use 'else' block to continue code if connection establishes successfully
else:
    print("Connection established successfully")

# Create a cursor object
cursor = conn.cursor()

# Write a SQL query to pull all data from database as is
all_data_query = "SELECT * FROM global_warming_data;"

# Execute the above SQL query
cursor.execute(all_data_query)

# Fetch all rows from the result
rows = cursor.fetchall()

# Use a for loop to pull column names from 'cursor description'
column_names = [desc[0] for desc in cursor.description]

# Create a new dataframe using the extracted data
gw_df = pd.DataFrame(rows, columns=column_names)
gw_df = gw_df.drop(columns=['record_id'])

# Close the cursor and the connection
cursor.close()
conn.close()

# Display the dataframe
gw_df.head(10)

Connection established successfully


,country,year,temperature_anomaly,co2_emissions,population,forest_area,gdp,renewable_energy_usage,methane_emissions,sea_level_rise,...,waste_management,per_capita_emissions,industrial_activity,air_pollution_index,biodiversity_index,ocean_acidification,fossil_fuel_usage,energy_consumption_per_capita,policy_score,average_temperature
0,Country_103,1913,-1.163537,8.876061e+08,1.627978e+08,54.872178,6.139887e+12,76.710013,8.317626e+06,8.111839,...,82.691409,2.285351,4.060975,150.285539,90.073356,8.025470,39.163860,1480.164332,78.870012,20.825292
1,Country_180,1950,-0.432122,4.497517e+08,4.281359e+08,84.051006,2.601447e+12,68.450021,6.206540e+06,42.025915,...,59.322883,17.411668,85.300604,27.305922,88.289837,8.021719,28.252554,1482.730048,32.600905,28.720587
2,Country_93,2014,0.444954,4.579080e+08,4.926732e+08,72.295357,5.192677e+12,36.725699,1.056885e+06,20.953840,...,94.982931,12.039703,83.804880,216.911429,86.936256,7.647408,61.548382,706.918809,37.671300,15.014084
3,Country_15,2020,-1.171616,5.049503e+08,1.252169e+09,17.259684,8.252128e+12,77.547901,1.986813e+06,45.599595,...,62.064250,2.853957,47.014265,35.869182,44.904331,7.569353,82.423750,2616.238324,86.581725,-1.277086
4,Country_107,1964,-0.564038,6.898891e+08,2.932960e+08,44.438605,8.560746e+12,10.019576,3.313252e+06,7.652150,...,84.431279,19.801173,89.379613,284.263093,8.102916,8.015415,29.964450,4975.683780,20.618406,2.861989
5,Country_72,1926,-1.946218,2.394448e+08,1.441203e+07,6.117781,1.989459e+12,26.163399,8.218906e+06,13.574102,...,36.112468,8.528017,63.807031,172.964007,20.512193,7.874246,67.153067,267.465131,84.672317,37.889804
6,Country_189,1921,-1.362100,1.977445e+08,1.339563e+09,75.554131,9.131238e+12,82.661868,5.160613e+06,1.378949,...,1.237686,7.484364,15.002742,238.288021,67.988814,8.448967,22.046013,3114.698772,72.740043,-7.786539
7,Country_21,1921,1.627624,2.111461e+08,2.410014e+07,25.999981,4.034784e+12,36.773252,5.890986e+06,15.617050,...,90.687488,15.747481,86.194366,150.058727,44.592418,8.436721,58.698935,3519.096247,19.243964,4.206395
8,Country_103,1989,0.541387,7.802026e+08,3.855137e+08,17.797781,6.823672e+12,3.422548,1.313058e+04,36.226933,...,60.895322,10.568785,66.175268,242.089767,93.935792,8.224373,46.630095,2718.348573,89.810124,-5.829195
9,Country_122,1963,0.016534,5.397029e+08,5.508798e+08,8.948443,4.713484e+12,72.875002,5.766624e+06,8.016881,...,77.485505,13.919414,70.265639,153.191500,77.122932,8.190353,52.587813,1745.691747,30.215953,2.473330


In [8]:
# Save to a new csv file
gw_df.to_csv('Resources/global_warming_data_sql.csv', index=False)

In [9]:
# Use a 'try' statement to attempt to conenct to database and to handle errors that occur
try:
    # Connect to the database
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=user,
        password=password,
        database="global_warming_db"
    )

# Use an 'except' block to handle error connecting to database    
except psycopg2.Error as e:
    print("Error connecting to the database:")
    print(e)

# Use 'else' block to continue code if connection establishes successfully
else:
    print("Connection established successfully")

# Create a cursor object
cursor = conn.cursor()

# Write a SQL query to pull all data from database as is
all_data_query = """
SELECT country, year,
AVG(temperature_anomaly) AS temperature_anomaly,
AVG(co2_emissions) AS co2_emissions,
AVG(population) AS population,
AVG(forest_area) AS forest_area,
AVG(gdp) AS gdp,
AVG(renewable_energy_usage) AS renewable_energy_usage,
AVG(methane_emissions) AS methane_emissions,
AVG(sea_level_rise) AS sea_level_rise,
AVG(artic_ice_extent) AS artic_ice_extent,
AVG(urbanization) AS urbanization,
AVG(deforestation_rate) AS deforestation_rate,
AVG(extreme_weather_events) AS extreme_weather_events,
AVG(average_rainfall) AS average_rainfall,
AVG(solar_energy_potential) AS solar_energy_potential,
AVG(waste_management) AS waste_management,
AVG(per_capita_emissions) AS per_capita_emissions,
AVG(industrial_activity) AS industrial_activity,
AVG(air_pollution_index) AS air_pollution_index,
AVG(biodiversity_index) AS biodiversity_index,
AVG(ocean_acidification) AS ocean_acidification,
AVG(fossil_fuel_usage) AS fossil_fuel_usage,
AVG(energy_consumption_per_capita) AS energy_consumption_per_capita,
AVG(policy_score) AS policy_score,
AVG(average_temperature) AS average_temperature
FROM global_warming_data
GROUP BY country, year;
"""

# Execute the above SQL query
cursor.execute(all_data_query)

# Fetch all rows from the result
rows = cursor.fetchall()

# Use a for loop to pull column names from 'cursor description'
column_names = [desc[0] for desc in cursor.description]

# Create a new dataframe using the extracted data
gw_cleaned_df = pd.DataFrame(rows, columns=column_names)

# Close the cursor and the connection
cursor.close()
conn.close()

# Display the dataframe
gw_cleaned_df.head(10)

Connection established successfully


,country,year,temperature_anomaly,co2_emissions,population,forest_area,gdp,renewable_energy_usage,methane_emissions,sea_level_rise,...,waste_management,per_capita_emissions,industrial_activity,air_pollution_index,biodiversity_index,ocean_acidification,fossil_fuel_usage,energy_consumption_per_capita,policy_score,average_temperature
0,Country_1,1900,-0.335027,3.984644e+08,3.750466e+08,27.856810,4.573252e+12,60.185651,5.169077e+06,24.478590,...,73.044383,10.581177,57.202100,160.881364,45.789450,8.079238,52.427376,2657.026661,42.942318,5.502509
1,Country_1,1901,0.170373,8.440511e+08,1.001558e+09,69.848395,4.868018e+12,39.525191,2.619170e+06,11.040926,...,50.237963,10.221387,27.407770,130.857790,51.708924,8.149605,76.355544,3506.219973,16.954388,19.801916
2,Country_1,1902,0.448391,7.090039e+08,3.604418e+08,50.116560,5.464041e+12,71.867926,6.380284e+06,27.972579,...,42.104585,6.778345,59.227467,166.268018,73.520410,7.903046,57.531928,2164.926898,48.193189,18.555981
3,Country_1,1903,1.254878,7.388654e+08,9.164562e+08,68.083451,4.415206e+12,45.673511,5.175568e+06,13.719263,...,18.991930,5.860874,48.448145,189.446269,55.501725,8.021785,45.270020,2062.859869,49.178483,25.610792
4,Country_1,1904,0.455433,5.804757e+08,4.870284e+08,21.907529,5.453627e+12,55.241938,3.343214e+06,11.041168,...,53.156173,7.771085,65.220926,117.647816,57.842259,7.875292,39.504759,3453.870114,34.840441,16.345709
5,Country_1,1905,0.410746,5.777469e+08,7.694415e+08,56.926024,5.780117e+12,68.232101,7.216920e+06,14.282445,...,53.011984,10.536067,69.161105,139.574633,61.011823,7.840244,60.622457,2953.886563,44.067380,13.847646
6,Country_1,1906,0.908977,7.024640e+08,7.358692e+08,71.332127,5.664276e+12,42.958334,4.902899e+06,5.448860,...,41.175097,13.095437,78.475460,116.944604,73.824494,7.992787,66.976307,1844.869843,52.498015,12.256493
7,Country_1,1907,-0.603829,6.040024e+08,8.475305e+08,45.097945,6.526098e+12,50.626980,3.227840e+06,28.496382,...,51.536035,8.826178,57.821706,133.100729,47.308481,7.891710,59.522495,2394.851237,62.134645,27.827221
8,Country_1,1908,0.177539,3.718096e+08,9.193322e+08,38.384588,7.034924e+12,47.509776,3.091091e+06,33.475862,...,43.829207,8.980073,30.278116,165.382946,64.354918,8.036241,41.874460,2508.433358,55.284421,13.713517
9,Country_1,1909,0.878918,3.146844e+08,1.010018e+09,45.963693,7.712606e+12,44.302738,2.117232e+06,34.444054,...,67.092046,8.013607,31.661106,68.882352,31.644136,8.176225,56.151403,3032.925004,21.075502,20.954948


In [10]:
# Save to a new csv file
gw_cleaned_df.to_csv('Resources/global_warming_data_sql_cleaned.csv', index=False) 